In [1]:
import matplotlib.pyplot as plt
# 한글 폰트 지정
plt.rc("font", family = "Malgun Gothic")
# 스타일 서식 train
plt.style.use('ggplot')

# 마이너스 기호 표시 설정
plt.rcParams['axes.unicode_minus'] = False

In [28]:
# 필요한 라이브러리 임포트

# 데이터 처리 및 시각화 라이브러리
import pandas as pd
import seaborn as sns
import numpy as np
import random
import os
from matplotlib.pylab import rcParams 

# 사이킷런 (scikit-learn) 라이브러리
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV  
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error, r2_score  
from sklearn.cluster import KMeans
from sklearn.tree import export_graphviz, DecisionTreeClassifier

# 머신러닝 라이브러리 (LightGBM, XGBoost)
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import plot_importance, XGBClassifier, plot_tree

In [6]:
train_df = pd.read_csv("C:\\Users\\USER\\projects\\5week\\project\\train_m.csv")

In [3]:
sample = pd.read_csv("C:\\Users\\USER\\projects\\5week\\project\\sample_submission.csv")
test = pd.read_csv("C:\\Users\\USER\\projects\\5week\\project\\test_m.csv")
train = pd.read_csv("C:\\Users\\USER\\projects\\5week\\project\\train_m.csv")
original_columns = test.columns.tolist()

In [4]:
# test데이터셋에 있는 변수만 선택
train = train[test.columns.to_list()+['Y_LABEL','SAMPLE_TRANSFER_DAY']].copy()

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14095 entries, 0 to 14094
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   14095 non-null  object 
 1   COMPONENT_ARBITRARY  14095 non-null  object 
 2   ANONYMOUS_1          14095 non-null  int64  
 3   YEAR                 14095 non-null  int64  
 4   SAMPLE_TRANSFER_DAY  14095 non-null  int64  
 5   ANONYMOUS_2          14095 non-null  int64  
 6   AG                   14095 non-null  int64  
 7   AL                   14095 non-null  int64  
 8   B                    14095 non-null  int64  
 9   BA                   14095 non-null  int64  
 10  BE                   14095 non-null  int64  
 11  CA                   14095 non-null  int64  
 12  CD                   12701 non-null  float64
 13  CO                   14095 non-null  int64  
 14  CR                   14095 non-null  int64  
 15  CU                   14095 non-null 

In [8]:
train_2= train_df[train_df['COMPONENT_ARBITRARY'] == 'COMPONENT2']

In [9]:
train_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2316 entries, 1 to 14094
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2316 non-null   object 
 1   COMPONENT_ARBITRARY  2316 non-null   object 
 2   ANONYMOUS_1          2316 non-null   int64  
 3   YEAR                 2316 non-null   int64  
 4   SAMPLE_TRANSFER_DAY  2316 non-null   int64  
 5   ANONYMOUS_2          2316 non-null   int64  
 6   AG                   2316 non-null   int64  
 7   AL                   2316 non-null   int64  
 8   B                    2316 non-null   int64  
 9   BA                   2316 non-null   int64  
 10  BE                   2316 non-null   int64  
 11  CA                   2316 non-null   int64  
 12  CD                   1980 non-null   float64
 13  CO                   2316 non-null   int64  
 14  CR                   2316 non-null   int64  
 15  CU                   2316 non-null   int64

In [11]:
train = train_2

In [12]:
# 각 열의 null 값 개수 계산
null_counts = train.isnull().sum()

# null 값 개수 출력
print(null_counts)

ID                        0
COMPONENT_ARBITRARY       0
ANONYMOUS_1               0
YEAR                      0
SAMPLE_TRANSFER_DAY       0
ANONYMOUS_2               0
AG                        0
AL                        0
B                         0
BA                        0
BE                        0
CA                        0
CD                      336
CO                        0
CR                        0
CU                        0
FH2O                   2316
FNOX                   2316
FOPTIMETHGLY           2316
FOXID                  2316
FSO4                   2316
FTBN                   2316
FE                        0
FUEL                   2316
H2O                       0
K                       200
LI                        0
MG                        0
MN                        0
MO                        0
NA                        0
NI                        0
P                         0
PB                        0
PQINDEX                   0
S                   

## 변수 드랍

In [13]:
train = train.drop(columns=['ID','COMPONENT_ARBITRARY','FH2O','FNOX','FOPTIMETHGLY','FOXID','FSO4','FTBN','SOOTPERCENTAGE'])

In [14]:
#U14, U6, U4 는 0으로 결측치 채우기
train[['U14', 'U6', 'U4']] = train[['U14', 'U6', 'U4']].fillna(0)

print(train)

       ANONYMOUS_1  YEAR  SAMPLE_TRANSFER_DAY  ANONYMOUS_2  AG   AL    B  BA  \
1             1350  2021                   51          375   0    2   19   0   
2             2415  2015                    2          200   0  110    1   1   
15            1830  2015                   13          550   0    1   19   2   
16            5694  2009                    6          200   0    1    2   0   
27            1706  2012                    7          200   0    2   10   0   
...            ...   ...                  ...          ...  ..  ...  ...  ..   
14079         1350  2017                    7          477   0    8    0   2   
14085         4976  2013                    5          200   0    0    1   0   
14089         1552  2007                    5          375   0    4  111   0   
14093         2498  2009                   19          550   0    2    4   0   
14094         1902  2012                   15          200   0    0   77   0   

       BE    CA  ...     U25     U20   

In [16]:
train = train.dropna()

In [18]:
train_df_counts = train_2['Y_LABEL'].value_counts()

train_counts = train['Y_LABEL'].value_counts()

print(train_df_counts)
print(train_counts)

# Define the counts for the two sets
counts_set1 = {'0': 3578, '1': 312}
counts_set2 = {'0': 2699, '1': 215}

# Calculate the percentages for each set
percent_set1 = {label: (count / sum(counts_set1.values())) * 100 for label, count in counts_set1.items()}
percent_set2 = {label: (count / sum(counts_set2.values())) * 100 for label, count in counts_set2.items()}

print(percent_set1)
print(percent_set2)

Y_LABEL
0    2110
1     206
Name: count, dtype: int64
Series([], Name: count, dtype: int64)
{'0': 91.97943444730076, '1': 8.020565552699228}
{'0': 92.62182566918325, '1': 7.378174330816746}


## x,y 분할

In [19]:
X = train.drop('Y_LABEL', axis=1)
y = train['Y_LABEL']

## automl

In [40]:
pip install --upgrade scipy

  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.21.1 requires pandas<2.1.0,>=1.1.0, but you have pandas 2.2.2 which is incompatible.
pycaret 3.1.0 requires pandas<2.0.0,>=1.3.0, but you have pandas 2.2.2 which is incompatible.
pycaret 3.1.0 requires scipy~=1.10.1, but you have scipy 1.13.1 which is incompatible.
You should consider upgrading via the 'c:\Users\USER\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [34]:
pip install pycaret

  Using cached pandas-1.5.3-cp39-cp39-win_amd64.whl (10.9 MB)
  Using cached scipy-1.10.1-cp39-cp39-win_amd64.whl (42.5 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\USER\\AppData\\Roaming\\Python\\Python39\\site-packages\\~cipy.libs\\libopenblas_v0.3.27--3aa239bc726cfb0bd8e5330d8d4c15c6.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\USER\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [35]:
pip show scipy

Name: scipy
Version: 1.10.1
Summary: Fundamental algorithms for scientific computing in Python
Home-page: https://scipy.org/
Author: None
Author-email: None
License: Copyright (c) 2001-2002 Enthought, Inc. 2003-2022, SciPy Developers.
        All rights reserved.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions
        are met:
        
        1. Redistributions of source code must retain the above copyright
           notice, this list of conditions and the following disclaimer.
        
        2. Redistributions in binary form must reproduce the above
           copyright notice, this list of conditions and the following
           disclaimer in the documentation and/or other materials provided
           with the distribution.
        
        3. Neither the name of the copyright holder nor the names of its
           contributors may be used to endorse or promote products

In [1]:
%pip uninstall scipy
%pip install scipy==1.7.3

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip uninstall pycaret
%pip install pycaret

In [2]:
from pycaret.classification import *

ImportError: cannot import name 'interp' from 'scipy' (c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py)

In [42]:
clf = setup(data, target='Y_LABEL', session_id=123, ignore_features=['ID'], fold=5, verbose=False)


NameError: name 'setup' is not defined

In [24]:
# PyCaret 불러오기
from pycaret.classification import *


# PyCaret 설정
clf_setup = setup(train, target='Y_LABEL', session_id=123)

ImportError: cannot import name 'interp' from 'scipy' (C:\Users\USER\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py)

In [ ]:
# 모델 비교
best_model = compare_models()

# 가장 성능이 좋은 모델 출력
print(best_model)

In [ ]:
# 모델 튜닝
tuned_model = tune_model(best_model)

# 튜닝된 모델 출력
print(tuned_model)

In [ ]:
# 모델 평가
evaluate_model(tuned_model)

In [ ]:
# 예측
predictions = predict_model(tuned_model)

# 모델 저장
save_model(tuned_model, 'best_tuned_model')

In [ ]:
# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)